# Mode choice calibration 

- Read original coeff csv file
- Change +- 20% of coefficients 
- Save change in a new csv file
- Update yaml file with new file name

In [240]:
import pandas as pd 
import yaml

In [241]:
coeffs = pd.read_csv('configs/configs/tour_mode_choice_coeffs.csv', 
                     index_col = 'Expression')

coeff_t = coeffs.drop(columns= 'atwork').T
at_work_coeff = coeffs['atwork'].reset_index(drop = True)

In [242]:
# coeff_t['c_ivt'] = 1.2 * coeff_t['c_ivt'].astype(float)

# Walk access and egress. #20% greater For walk to transit options
c_ivt_coeff = ['c_wacc', 'c_wegr']
for col_name in c_ivt_coeff:
    # Split str colum in float and str part. 
    split = coeff_t[col_name].str.split('*', expand = True) 
    split[0] = (split[0].astype(float)*1.2).astype(str)
    new_col= split[0] + '*' + split[1] 
    coeff_t[col_name] = new_col
    
# Walking time: Modify coefficient by 20%
c_ivt_coeff = ['c_walktimeshort', 'c_walktimelong']
for col_name in c_ivt_coeff:
    # Split str colum in float and str part. 
    split = coeff_t[col_name].str.split('*', expand = True) 
    split[0] = (split[0].astype(float)*0.8).astype(str)
    new_col= split[0] + '*' + split[1] 
    coeff_t[col_name] = new_col

In [243]:
# Modify constants 

# NonMontorized and Auto ASC. 20% Greater
walk_asc = ['walk_ASC_no_auto', 'walk_ASC_auto_decicient', 'walk_ASC_auto_sufficient',
            'joint_walk_ASC_no_auto', 'joint_walk_ASC_auto_decicient', 
            'joint_walk_ASC_auto_sufficient', 'sr2_ASC_auto_decicient',
            'sr2_ASC_auto_sufficient', 'sr3p_ASC_no_auto', 
            'sr3p_ASC_auto_decicient','sr3p_ASC_auto_sufficient']
for col_name in walk_asc:
    col = coeff_t[col_name].astype(float)
    col.where(col<0,col*1.2, inplace = True)  
    col.where(col>0,col*0.8, inplace = True) 
    coeff_t[col_name] = col
    

# Walk and drive to transit ASC:
walk_drive_transit_asc = ['walk_transit_ASC_no_auto', 
                          'walk_transit_ASC_auto_decicient',
                          'walk_transit_ASC_auto_sufficient', 
                          'drive_transit_ASC_auto_decicient',
                          'drive_transit_ASC_auto_sufficient',
                          'walk_transit_CBD_ASC', 'drive_transit_CBD_ASC',
                          'local_bus_ASC', 'joint_walk_transit_ASC_no_auto',
                          'joint_walk_transit_ASC_auto_decicient',
                          'joint_walk_transit_ASC_auto_sufficient',
                          'joint_drive_transit_ASC_auto_decicient',
                          'joint_drive_transit_ASC_auto_sufficient']

#Greater modification to account for waiting and transfer time 
#TO DO: Modify when better waiting and tranfer skims are available. 
for col_name in walk_drive_transit_asc:
    col = coeff_t[col_name].astype(float)
    col.where(col>0,col*1.5, inplace = True)
    col.where(col<0,col*0.7, inplace = True)
    coeff_t[col_name] = col

In [244]:
#Transpose and append 'atwork' coefficients
coeff_final = coeff_t.T.reset_index()
coeff_final['atwork'] = at_work_coeff

#Saves files as V2
new_name = 'tour_mode_choice_coeffs_v2.csv'
coeff_final.to_csv('configs/configs/'+ new_name, index = False)

## Modify YAML file

In [245]:
#Tour .yaml file 
yaml_file = open('configs/configs/tour_mode_choice.yaml')
model_settings = yaml.load(yaml_file, Loader=yaml.FullLoader)

# Change CostPerMile 
model_settings['CONSTANTS']['costPerMile'] = 18.29 * 0.8 #Reduce cost per mile

#Walk speed and treshold
model_settings['CONSTANTS']['walkThresh'] = 2 # Before 1.5
model_settings['CONSTANTS']['walkSpeed'] = 3.5 #Before 3.0

#Modify Coeffs.csv file 
model_settings['COEFFS'] = new_name

#Saves File 
with open('configs/configs/tour_mode_choice.yaml', 'w') as file:
    documents = yaml.dump(model_settings, file)

In [246]:
#Trips .yaml file 
yaml_file = open('configs/configs/trip_mode_choice.yaml')
model_settings = yaml.load(yaml_file, Loader=yaml.FullLoader)

model_settings['CONSTANTS']['costPerMile'] = 18.29 * 0.8 #Reduce cost per mile
model_settings['CONSTANTS']['walkThresh'] = 2 # Before 1.5
model_settings['CONSTANTS']['walkSpeed'] = 3.5 #Before 3.0

#Saves file
with open('configs/configs/trip_mode_choice.yaml', 'w') as file:
    documents = yaml.dump(model_settings, file)